# Imports

In [ ]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Add, Reshape
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import plot_model

from nltk.translate.bleu_score import corpus_bleu

In [ ]:
from dataset.utils import load_coco, load_image, print_progress_bar
from image_model.topic_layers import load_topic_model, load_feature_model

# Load Data

In [ ]:
# Folder containing the datset
data_dir = 'dataset/'

In [ ]:
train_data, val_data, test_data, category_id, id_category = load_coco(
    os.path.join(data_dir, 'coco_raw.pickle'), 'captions'
)

In [ ]:
train_images, train_captions = train_data  # Load training data
val_images, val_captions = val_data  # Load validation data
test_images, test_captions = test_data  # Load test data

In [ ]:
num_classes = len(id_category)
num_classes

In [ ]:
num_images_train = len(train_images)
num_images_train

In [ ]:
num_images_val = len(val_images)
num_images_val

### Display a sample image

In [ ]:
# display image
idx = 10

for caption in train_captions[idx]:
    print(caption)

plt.imshow(load_image(os.path.join(data_dir, train_images[idx])))
plt.show()

# Load Pre-Trained Image Model

In [ ]:
# Load the pre-trained topic model
weights_path = os.path.join(os.getcwd(), 'topic_extraction', 'weights', 'checkpoint.keras')
topic_model = load_topic_model(num_classes, weights_path)
topic_model.summary()

In [ ]:
# Load the pre-trained feature extractor model
feature_model = load_feature_model()

### Process Images

In [ ]:
def process_images(topic_model, feature_model, data_dir, filenames, batch_size):
    """
    Process all the given files in the given data_dir using the
    pre-trained topic-model as well as the feature-model and return
    their transfer-values.
    
    The images are processed in batches to save
    memory and improve efficiency.
    """
    
    num_images = len(filenames)
    img_size = K.int_shape(topic_model.input)[1:3]    # Expected input size of the pre-trained network

    # Pre-allocate input-batch-array for images.
    shape = (batch_size,) + img_size + (3,)
    image_batch = np.zeros(shape=shape, dtype=np.float32)

    # Pre-allocate output-array for transfer-values.
    topic_transfer_values = np.zeros(
        shape=(num_images,) + K.int_shape(topic_model.output)[1:],
        dtype=np.float32
    )
    feature_transfer_values = np.zeros(
        shape=(num_images, K.int_shape(feature_model.output)[1]),
        dtype=np.float32
    )

    start_index = 0
    print_progress_bar(start_index, num_images)  # Initial call to print 0% progress

    while start_index < num_images:
        end_index = start_index + batch_size
        if end_index > num_images:
            end_index = num_images
        current_batch_size = end_index - start_index

        # Load all the images in the batch.
        for i, filename in enumerate(filenames[start_index:end_index]):
            path = os.path.join(data_dir, filename)
            img = load_image(path, size=img_size)
            image_batch[i] = img

        # Use the pre-trained models to process the image.
        topic_transfer_values_batch = topic_model.predict(
            image_batch[0:current_batch_size]
        )
        feature_transfer_values_batch = feature_model.predict(
            image_batch[0:current_batch_size]
        )

        # Save the transfer-values in the pre-allocated arrays.
        topic_transfer_values[start_index:end_index] = topic_transfer_values_batch[0:current_batch_size]
        feature_transfer_values[start_index:end_index] = feature_transfer_values_batch[0:current_batch_size]

        start_index = end_index
        print_progress_bar(start_index, num_images)  # Update Progress Bar

    print()
    return topic_transfer_values, feature_transfer_values

In [ ]:
def process_data(topic_model, feature_model, data_dir, data_type, filenames, captions, batch_size):
    print('Processing {0} images in {1}-set ...'.format(len(filenames), data_type))

    # Path for the cache-file.
    cache_path_dir = os.path.join(data_dir, 'processed_caption_data')
    topic_cache_path = os.path.join(
        cache_path_dir, 'topic_transfer_values_{}.pkl'.format(data_type)
    )
    feature_cache_path = os.path.join(
        cache_path_dir, 'feature_transfer_values_{}.pkl'.format(data_type)
    )
    images_cache_path = os.path.join(
        cache_path_dir, 'images_{}.pkl'.format(data_type)
    )
    captions_cache_path = os.path.join(
        cache_path_dir, 'captions_{}.pkl'.format(data_type)
    )
    
    # Check if directory to store processed data exists
    if not os.path.exists(cache_path_dir):
        print('Directory created:', cache_path_dir)
        os.mkdir(cache_path_dir)

    # If the cache-file already exists then reload it,
    # otherwise process all images and save their transfer-values
    # to the cache-file so it can be reloaded quickly.
    topic_path_exists = os.path.exists(topic_cache_path)
    feature_path_exists = os.path.exists(feature_cache_path)
    image_path_exists = os.path.exists(images_cache_path)
    caption_path_exists = os.path.exists(captions_cache_path)
    if topic_path_exists and feature_path_exists and image_path_exists and caption_path_exists:
        with open(topic_cache_path, mode='rb') as file:
            topic_obj = pickle.load(file)
        with open(feature_cache_path, mode='rb') as file:
            feature_obj = pickle.load(file)
        with open(images_cache_path, mode='rb') as file:
            images = pickle.load(file)
        with open(captions_cache_path, mode='rb') as file:
            captions = pickle.load(file)
        print("Data loaded from cache-file.")
    else:
        topic_obj, feature_obj = process_images(
            topic_model, feature_model, data_dir, filenames, batch_size
        )
        with open(topic_cache_path, mode='wb') as file:
            pickle.dump(topic_obj, file)
        with open(feature_cache_path, mode='wb') as file:
            pickle.dump(feature_obj, file)
        with open(images_cache_path, mode='wb') as file:
            pickle.dump(filenames, file)
        with open(captions_cache_path, mode='wb') as file:
            pickle.dump(captions, file)
        print("Data saved to cache-file.")

    return topic_obj, feature_obj, images, captions

In [ ]:
process_batch_size = 128

In [ ]:
%%time
# Training Data
topic_transfer_values_train, feature_transfer_values_train, images_train, captions_train = process_data(
    topic_model, feature_model, data_dir, 'train', train_images, train_captions, process_batch_size
)
print("topic shape:", topic_transfer_values_train.shape)
print("feature shape:", feature_transfer_values_train.shape)

In [ ]:
%%time
# Validation Data
topic_transfer_values_val, feature_transfer_values_val, images_val, captions_val = process_data(
    topic_model, feature_model, data_dir, 'val', val_images, val_captions, process_batch_size
)
print("topic shape:", topic_transfer_values_val.shape)
print("feature shape:", feature_transfer_values_val.shape)

In [ ]:
%%time
# Test Data
topic_transfer_values_test, feature_transfer_values_test, images_test, captions_test = process_data(
    topic_model, feature_model, data_dir, 'test', test_images, test_captions, process_batch_size
)
print("topic shape:", topic_transfer_values_test.shape)
print("feature shape:", feature_transfer_values_test.shape)

# Tokenizer

In [ ]:
mark_start = 'startseq'
mark_end = 'endseq'

In [ ]:
def mark_captions(captions_list):
    """ Mark all the captions with the start and the end marker """
    captions_marked = [
        [' '.join([mark_start, caption, mark_end]) for caption in captions] for captions in captions_list
    ]
    
    return captions_marked

In [ ]:
captions_train_marked = mark_captions(captions_train)
print('Marked captions:')
captions_train_marked[0]

In [ ]:
def flatten(captions_list):
    """ Flatten all the captions into a single list """
    caption_list = [caption
                    for caption_list in captions_list
                    for caption in caption_list]
    
    return caption_list

In [ ]:
captions_train_flat = flatten(captions_train_marked)

Create the tokenizer

In [ ]:
%%time
tokenizer = Tokenizer()
tokenizer.fit_on_texts(captions_train_flat)

In [ ]:
# Get integer token for the start marker
token_start = tokenizer.word_index[mark_start]
token_start

In [ ]:
# Get integer token for the end marker
token_end = tokenizer.word_index[mark_end]
token_end

In [ ]:
# Numbers of words in the vocabulary
vocab_size = len(tokenizer.word_index) + 1
vocab_size

In [ ]:
# Max length of each caption
max_tokens = 16

# Data Generator

In [ ]:
def create_sequences(tokenizer, max_length, topic_transfer_value, feature_transfer_value, caption):
    """ Create sequences of topic_values, feature_values, input sequence and output sequence for an image """
    topic_values, feature_values = [], []
    input_captions, output_captions = [], []
    integer_sequence = tokenizer.texts_to_sequences([caption])[0]  # encode the sequence
    
    for idx in range(1, len(integer_sequence)):
        in_seq, out_seq = integer_sequence[:idx], integer_sequence[idx]  # split into input and output pair
        in_seq = pad_sequences([in_seq], maxlen=max_length, padding='post', truncating='post')[0]  # pad input sequence
        out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]  # encode output sequence
        
        # store
        topic_values.append(topic_transfer_value)
        feature_values.append(feature_transfer_value)
        input_captions.append(in_seq)
        output_captions.append(out_seq)
        
    return topic_values, feature_values, input_captions, output_captions

In [ ]:
def batch_generator(topic_transfer_values, feature_transfer_values, captions_list, tokenizer, num_images, batch_size, max_length, vocab_size):
    """
    Generator function for creating random batches of training-data.
    
    It selects the data completely randomly for each
    batch, corresponding to sampling of the training-set with
    replacement. This means it is possible to sample the same
    data multiple times within a single epoch - and it is also
    possible that some data is not sampled at all within an epoch.
    However, all the data should be unique within a single batch.
    """

    # Infinite loop.
    while True:
        # Get a list of random indices for images in the dataset.
        indices = np.random.randint(num_images, size=batch_size)
        
        # For a batch of the randomly chosen images there are
        # at least 5 captions describing the contents of the image.
        # Select one of those captions at random
        topic_values, feature_values = [], []
        input_captions, output_captions = [], []
        for idx in indices:
            topic_value, feature_value, input_caption, output_caption = create_sequences(
                tokenizer,
                max_length,
                topic_transfer_values[idx],
                feature_transfer_values[idx],
                np.random.choice(captions_list[idx])
            )
            topic_values.extend(topic_value)
            feature_values.extend(feature_value)
            input_captions.extend(input_caption)
            output_captions.extend(output_caption)

        # Dict for the input-data. Because we have
        # several inputs, we use a named dict to
        # ensure that the data is assigned correctly.
        x_data = {
            'caption_input': np.array(input_captions),
            'topic_input': np.array(topic_values),
            'feature_input': np.array(feature_values)
        }

        # Dict for the output-data.
        y_data = {
            'caption_output': np.array(output_captions)
        }
        
        yield (x_data, y_data)

In [ ]:
batch_size = 10

In [ ]:
generator_train = batch_generator(
    topic_transfer_values_train,
    feature_transfer_values_train,
    captions_train_marked,
    tokenizer,
    num_images_train,
    batch_size,
    max_tokens,
    vocab_size
)
batch = next(generator_train)
batch_x = batch[0]
batch_y = batch[1]

In [ ]:
# Example of the topic-transfer-values for the first image in the batch
batch_x['topic_input'][0]

In [ ]:
# Example of the feature-transfer-values for the first image in the batch
batch_x['feature_input'][0]

In [ ]:
# Example of the token-sequence for the first image in the batch
# This is the input to the decoder-part of the neural network
batch_x['caption_input'][0]

In [ ]:
# This is the token-sequence for the output of the decoder
# Note how it is the same as the sequence above, except it is shifted one time-step
batch_y['caption_output'][0]

In [ ]:
# Shape of the output
batch_y['caption_output'].shape

## Steps Per Epoch

One epoch is a complete processing of the training-set. We would like to process each image and caption pair only once per epoch. However, because each batch is chosen completely at random in the above batch-generator, it is possible that an image occurs in multiple batches within a single epoch, and it is possible that some images may not occur in any batch at all within a single epoch.

Nevertheless, we still use the concept of an 'epoch' to measure approximately how many iterations of the training-data we have processed. But the data-generator will generate batches for eternity, so we need to manually calculate the approximate number of batches required per epoch.

In [ ]:
# Number of captions for each image in the training-set
num_captions_train = [len(captions) for captions in captions_train]

# Total number of captions in the training-set
total_num_captions_train = np.sum(num_captions_train)

# Approximate number of batches required per epoch,
# if we want to process each caption and image pair once per epoch
steps_per_epoch_train = int(total_num_captions_train / batch_size)
steps_per_epoch_train

# Create Model

### Create pre-trained Embedding Layer

In [ ]:
def read_glove_vecs(glove_file):
    print('Creating word to vec map...')
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float32)
    print('Done!')
    return word_to_vec_map

In [ ]:
# load embeddings
word_to_vec_map = read_glove_vecs('{}/glove.6B.300d.txt'.format(data_dir))

In [ ]:
# assign embeddings values to custom tokens
size = word_to_vec_map['unk'].shape

word_to_vec_map[mark_start] = np.random.uniform(low=-1.0, high=1.0, size=size)
word_to_vec_map[mark_end] = np.random.uniform(low=-1.0, high=1.0, size=size)

In [ ]:
def create_embedding_layer(word_to_index, word_to_vec_map, num_words):
    """ Create a Keras Embedding() layer and load in pre-trained GloVe 300-dimensional vectors
        @params:
        :word_to_index -- dictionary containing the each word mapped to its index
        :word_to_vec_map -- dictionary mapping words to their GloVe vector representation
        :num_words -- number of words in the vocabulary
        
        @return:
        :decoder_embedding -- pretrained layer Keras instance
    """
    
    vocabulary_length = num_words + 1  # adding 1 to fit Keras embedding (requirement)
    embedding_dimensions = word_to_vec_map['unk'].shape[0]  # define dimensionality of GloVe word vectors (= 300)
    
    embedding_matrix = np.zeros((vocabulary_length, embedding_dimensions))  # initialize with zeros
    for word, index in word_to_index.items():
        try:
            embedding_matrix[index, :] = word_to_vec_map[word]
        except KeyError:
            embedding_matrix[index, :] = word_to_vec_map['unk']
    
    # we don't want the embeddings to be updated, thus trainable parameter is set to False
    decoder_embedding = Embedding(
        input_dim=vocabulary_length,
        output_dim=embedding_dimensions,
        trainable=False,
        name='decoder_embedding'
    )
    decoder_embedding.build((None,))
    decoder_embedding.set_weights([embedding_matrix])  # with this the layer is now pretrained
    
    return decoder_embedding

### Define some global values

In [ ]:
# Internal state-sizes of LSTMs
state_size = 256

In [ ]:
K.clear_session()

## Encode Images

In [ ]:
# Input layer to receive the predictions from the feature model
feature_input = Input(
    shape=K.int_shape(feature_model.output)[1:], name='feature_input'
)

In [ ]:
# Reshape feature transfer values
image_model_output = Dense(state_size, activation='relu', name='image_model_output')(feature_input)
image_model_output

## Encode Captions

### Create model to encode captions

Define layers

In [ ]:
# This inputs topic-transfer-values to the LSTM
topic_input = Input(
    shape=K.int_shape(topic_model.output)[1:], name='topic_input'
)

In [ ]:
# Input for token-sequences to the decoder
# Using 'None' in the shape means that the token-sequences can have arbitrary lengths
caption_input = Input(shape=(max_tokens,), name='caption_input')

In [ ]:
# Embedding-layer which converts sequences of integer-tokens to sequences of vectors
caption_embedding = create_embedding_layer(tokenizer.word_index, word_to_vec_map, vocab_size)

In [ ]:
# The LSTM layer for the input captions
caption_lstm = LSTM(state_size, name='caption_lstm')

Connect the layers

In [ ]:
# Reshape the topic transfer values to 3D tensor in-order to feed it to the LSTM
topic_input_reshaped = Reshape(
    target_shape=(K.int_shape(topic_input)[1:] + (1,))
)(topic_input)

In [ ]:
# Feed topic to LSTM
_, initial_state_h0, initial_state_c0 = LSTM(
    state_size, return_state=True, name='topic_lstm'
)(topic_input_reshaped)

topic_lstm_states = [initial_state_h0, initial_state_c0]

In [ ]:
net = caption_input  # Start the decoder-network with its input-layer
net = caption_embedding(net)  # Connect the embedding-layer
caption_model_output = caption_lstm(net, initial_state=topic_lstm_states) # Connect the caption LSTM layer
caption_model_output

## Merge the image and the caption model

In [ ]:
# Concat the outputs of both the models
merge_net = Add()([image_model_output, caption_model_output])

In [ ]:
merge_net = Dense(state_size, activation='relu')(merge_net)
outputs = Dense(vocab_size, activation='softmax', name='caption_output')(merge_net)
outputs

# Create the Training Model

In [ ]:
model = Model(
    inputs=[feature_input, topic_input, caption_input],
    outputs=outputs
)

In [ ]:
model.summary()

## Compile the Training Model

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

### Callback Functions

In [ ]:
path_checkpoint = 'weights/checkpoint1.keras'
callback_checkpoint = ModelCheckpoint(filepath=path_checkpoint,
                                      verbose=1,
                                      save_weights_only=True)

In [ ]:
callback_tensorboard = TensorBoard(log_dir='./weights/logs/',
                                   histogram_freq=0,
                                   write_graph=False)

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

In [ ]:
callbacks = [callback_checkpoint, callback_tensorboard]

### Load Checkpoint

In [ ]:
try:
    model.load_weights(path_checkpoint)
    print('Weights loaded.')
except Exception as error:
    print("Error trying to load checkpoint.")
    print(error)

### Visualize model

In [ ]:
plot_model(model, to_file='caption_model.png', show_shapes=True, show_layer_names=True)

### Train the Model

In [ ]:
%%time
model.fit_generator(
    generator=generator_train,
    steps_per_epoch=steps_per_epoch_train,
    epochs=20,
    callbacks=callbacks
)

# Generate Captions

In [ ]:
def generate_captions(topic_values, feature_values, caption_model, tokenizer, max_tokens):
    # Start with the initial start token
    predicted_caption = 'startseq'
    
    # Input for the caption model
    x_data = {
        'topic_input': np.expand_dims(topic_values, axis=0),
        'feature_input': np.expand_dims(feature_values, axis=0)
    }
    
    for i in range(max_tokens):
        sequence = tokenizer.texts_to_sequences([predicted_caption])[0]
        sequence = pad_sequences([sequence], maxlen=max_tokens)
        
        # predict next word
        x_data['caption_input'] = np.array(sequence)
        y_pred = caption_model.predict(x_data, verbose=0)
        y_pred = np.argmax(y_pred)
        word = tokenizer.index_word[y_pred]
        
        # stop if we cannot map the word
        if word is None:
            break
            
        # append as input for generating the next word
        predicted_caption += ' ' + word
        
        # stop if we predict the end of the sequence
        if word == 'endseq':
            break
    
    return ' '.join(predicted_caption.split()[1:-1])

In [ ]:
idx = 12
image_path = 'dataset/' + images_test[idx]

In [ ]:
# Plot the image.
plt.imshow(load_image(image_path))
plt.show()

# Predicted Caption
predicted_caption = generate_captions(
    topic_transfer_values_test[idx], feature_transfer_values_test[idx], model, tokenizer, max_tokens
)
print('Predicted Caption:')
print(predicted_caption)

# Original Captions
print('Original Captions:')
print(captions_test[idx])

# Obtain BLEU Score

In [ ]:
def evaluate_model(topic_values, feature_values, caption_model, captions, tokenizer, max_tokens):
    actual, predicted = [], []
    captions_length = len(captions)
    
    # Initial call to print 0% progress
    print_progress_bar_counter = 0
    print_progress_bar(print_progress_bar_counter, captions_length)
    
    for idx in range(captions_length):
        # generate description
        y_pred = generate_captions(
            topic_values[idx], feature_values[idx], caption_model, tokenizer, max_tokens
        )
        
        # store actual and predicted
        references = [caption.split() for caption in captions[idx]]
        actual.append(references)
        predicted.append(y_pred.split())
        
        # Update Progress Bar
        print_progress_bar_counter += 1
        print_progress_bar(print_progress_bar_counter, captions_length)

    print()
    # calculate BLEU score
    print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
    print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
    print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

In [ ]:
%%time
evaluate_model(
    topic_transfer_values_val, feature_transfer_values_val, model, captions_val, tokenizer, max_tokens
)